In [1]:
import warnings


warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids

In [3]:
# Load the data
file_path = Path('project_cc.csv')
loop_df = pd.read_csv(file_path)

In [4]:
loop_df["New_Results"] = ""

In [5]:
loop_df["Payment_Ratio"] = ""

In [6]:
loop_df.rename(columns={'default.payment.next.month':'default_pay'}, inplace=True)
loop_df.rename(columns={'New_Results':'Total_Missed_Pymts'}, inplace=True)

In [7]:
loop_df['PAY_0'] = [0 if x <= 0 else 1 for x in loop_df['PAY_0']]
loop_df['PAY_2'] = [0 if x <= 0 else 1 for x in loop_df['PAY_2']]
loop_df['PAY_3'] = [0 if x <= 0 else 1 for x in loop_df['PAY_3']]
loop_df['PAY_4'] = [0 if x <= 0 else 1 for x in loop_df['PAY_4']]
loop_df['PAY_5'] = [0 if x <= 0 else 1 for x in loop_df['PAY_5']]
loop_df['PAY_6'] = [0 if x <= 0 else 1 for x in loop_df['PAY_6']]

# 0 = made payment and 1 = missed payment

In [8]:
loop_df.head(10)

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio
0,1,20000.0,1,24,1,1,0,0,0,0,...,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,,
1,2,120000.0,2,26,0,1,0,0,0,1,...,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,,
2,3,90000.0,2,34,0,0,0,0,0,0,...,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,,
3,4,50000.0,1,37,0,0,0,0,0,0,...,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,,
4,5,50000.0,1,57,0,0,0,0,0,0,...,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,,
5,6,50000.0,2,37,0,0,0,0,0,0,...,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0,,
6,7,500000.0,2,29,0,0,0,0,0,0,...,473944.0,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,0,,
7,8,100000.0,2,23,0,0,0,0,0,0,...,567.0,380.0,601.0,0.0,581.0,1687.0,1542.0,0,,
8,9,140000.0,1,28,0,0,1,0,0,0,...,3719.0,3329.0,0.0,432.0,1000.0,1000.0,1000.0,0,,
9,10,20000.0,2,35,0,0,0,0,0,0,...,13912.0,0.0,0.0,0.0,13007.0,1122.0,0.0,0,,


In [9]:
loop_df['Total_Missed_Pymts']=loop_df['PAY_0']+loop_df['PAY_2']+loop_df['PAY_3']+loop_df['PAY_4']+loop_df['PAY_5']+loop_df['PAY_6']

In [10]:
loop_df['Total_Bill_AMT']=loop_df['BILL_AMT1'] + loop_df['BILL_AMT2'] + loop_df['BILL_AMT3'] + loop_df['BILL_AMT4'] + loop_df['BILL_AMT5'] + loop_df['BILL_AMT6'] 

In [11]:
loop_df['Total_Payment']=loop_df['PAY_AMT1'] + loop_df['PAY_AMT2'] + loop_df['PAY_AMT3'] + loop_df['PAY_AMT4'] + loop_df['PAY_AMT5'] + loop_df['PAY_AMT6'] 


In [12]:
loop_df['%_Amount_Paid'] = loop_df['Total_Payment'] / loop_df['Total_Bill_AMT'] *100

In [13]:
loop_df['Payment_Ratio']= (loop_df['Total_Missed_Pymts'] / 6) *100

In [80]:
#loop_df.rename(columns={'default.payment.next.month':'default_pay'}, inplace=True)
#loop_df.rename(columns={'New_Results':'Total_Missed_Pymts'}, inplace=True)


In [14]:
loop_df.head()

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio,Total_Bill_AMT,Total_Payment,%_Amount_Paid
0,1,20000.0,1,24,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1,2,33.333333,7704.0,689.0,8.943406
1,2,120000.0,2,26,0,1,0,0,0,1,...,1000.0,1000.0,0.0,2000.0,1,2,33.333333,17077.0,5000.0,29.279147
2,3,90000.0,2,34,0,0,0,0,0,0,...,1000.0,1000.0,1000.0,5000.0,0,0,0.000000,101653.0,11018.0,10.838834
3,4,50000.0,1,37,0,0,0,0,0,0,...,1200.0,1100.0,1069.0,1000.0,0,0,0.000000,231334.0,8388.0,3.625926
4,5,50000.0,1,57,0,0,0,0,0,0,...,10000.0,9000.0,689.0,679.0,0,0,0.000000,109339.0,59049.0,54.005433


In [15]:
loop_df.corr()['default_pay']


ID                   -0.013952
LIMIT_BAL            -0.153520
MARRIAGE             -0.024339
AGE                   0.013890
PAY_0                 0.368117
PAY_2                 0.338022
PAY_3                 0.293583
PAY_4                 0.275495
PAY_5                 0.266959
PAY_6                 0.246089
BILL_AMT1            -0.019644
BILL_AMT2            -0.014193
BILL_AMT3            -0.014076
BILL_AMT4            -0.010156
BILL_AMT5            -0.006760
BILL_AMT6            -0.005372
PAY_AMT1             -0.072929
PAY_AMT2             -0.058579
PAY_AMT3             -0.056250
PAY_AMT4             -0.056827
PAY_AMT5             -0.055124
PAY_AMT6             -0.053183
default_pay           1.000000
Total_Missed_Pymts    0.398394
Payment_Ratio         0.398394
Total_Bill_AMT       -0.012691
Total_Payment        -0.102354
%_Amount_Paid        -0.011391
Name: default_pay, dtype: float64

In [16]:
X=loop_df[['MARRIAGE', 'AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','Total_Missed_Pymts', 'Payment_Ratio','Total_Payment']]
y=loop_df[['default_pay']]


# Important

I made a new csv sheet after I made the new colums for our dataframe. I call it clean_data. On future models you will see me refer to this as clean_df. This is the code that I used to make this.

loop_df.to_csv('clean_data.csv', encoding='utf-8', index=False)


In [151]:
#loop_df.to_csv('clean_data.csv', encoding='utf-8', index=False)


In [17]:
y['default_pay'].value_counts()

0    23364
1     6636
Name: default_pay, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [19]:
print(f'''% Positive class in Train ={np.round(y_train.value_counts(normalize=True)[1] *100, 2)}''')

% Positive class in Train =default_pay
1              22.08
dtype: float64


In [20]:
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [21]:
X_scaler = scaler.fit(X_train)


In [22]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [24]:
from sklearn.metrics import balanced_accuracy_score
lr_y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, lr_y_pred)

0.6170113292082383

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, lr_y_pred)

array([[5564,  268],
       [1201,  467]], dtype=int64)

In [26]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8071111111111111
Testing Data Score: 0.8041333333333334


In [27]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, lr_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.95      0.28      0.88      0.52      0.29      5832
          1       0.64      0.28      0.95      0.39      0.52      0.25      1668

avg / total       0.78      0.80      0.43      0.77      0.52      0.28      7500



### Naive Oversampler

In [128]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=1)
# resampling X, y
X_ros, y_ros = ros.fit_resample(X_train_scaled, y_train)
#ros_preds=ros.predict(X_test)
# new class distribution 
print(f''' Shape of X before ROS: { X_train_scaled.shape}
Shape of X after ROS: {X_ros.shape}''')

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_ros.value_counts(normalize=True) * 100)

 Shape of X before ROS: (22500, 10)
Shape of X after ROS: (35064, 10)


Counter({50.0: 2})

In [129]:
ros_model= LogisticRegression(solver='lbfgs', random_state=1)
ros_model.fit(X_ros, y_ros)

LogisticRegression(random_state=1)

In [130]:
ros_y_pred = ros_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, ros_y_pred)

0.6960851072228637

In [131]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, ros_y_pred)

array([[4850,  982],
       [ 733,  935]], dtype=int64)

In [132]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, ros_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.83      0.56      0.85      0.68      0.48      5832
          1       0.49      0.56      0.83      0.52      0.68      0.45      1668

avg / total       0.78      0.77      0.62      0.78      0.68      0.47      7500



# Smote #

In [135]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

sm=SMOTE(random_state=1)

Smote_X_resampled, Smote_y_resampled = sm.fit_resample(X_train_scaled, y_train)

print(f''' Shape of X before SMOTE: { X_train_scaled.shape}
Shape of X after SMOTE: {Smote_X_resampled.shape}''')

# View the count of target classes with Counter
# YOUR CODE HERE
Smote_y_resampled.value_counts(normalize=True) *100


 Shape of X before SMOTE: (22500, 10)
Shape of X after SMOTE: (35064, 10)


default_pay
1              50.0
0              50.0
dtype: float64

In [138]:
smote_ros_model= LogisticRegression(solver='lbfgs', random_state=1)
smote_ros_model.fit(Smote_X_resampled, Smote_y_resampled)


LogisticRegression(random_state=1)

In [139]:
smote_y_pred = smote_ros_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, smote_y_pred) 

0.6962565749869241

In [140]:
confusion_matrix(y_test, smote_y_pred)

array([[4852,  980],
       [ 733,  935]], dtype=int64)

In [141]:
print(classification_report_imbalanced(y_test, smote_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.83      0.56      0.85      0.68      0.48      5832
          1       0.49      0.56      0.83      0.52      0.68      0.45      1668

avg / total       0.78      0.77      0.62      0.78      0.68      0.47      7500



## Smoteen ##

In [142]:
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
smoteen_X_resampled, smoteen_y_resampled = sm.fit_resample(X_train_scaled, y_train)

In [143]:
Counter(smoteen_y_resampled.value_counts())

Counter({9017: 1, 5658: 1})

In [144]:
smoteen_model= LogisticRegression(solver='lbfgs', random_state=1)
smoteen_model.fit(smoteen_X_resampled, smoteen_y_resampled)

LogisticRegression(random_state=1)

In [145]:
smoteen_y_pred = smoteen_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, smoteen_y_pred)

0.6955275285944085

In [146]:
confusion_matrix(y_test, smoteen_y_pred)

array([[4840,  992],
       [ 732,  936]], dtype=int64)

In [147]:
print(classification_report_imbalanced(y_test, smoteen_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.83      0.56      0.85      0.68      0.48      5832
          1       0.49      0.56      0.83      0.52      0.68      0.45      1668

avg / total       0.78      0.77      0.62      0.78      0.68      0.47      7500

